In [1]:
import gensim
import pandas as pd
import numpy as np
import sys
import os
import itertools
import time

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

from scipy import stats
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans, MiniBatchKMeans

reload(sys)
sys.setdefaultencoding("UTF-8")

In [2]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [3]:
def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [4]:
# Define a function to create bags of centroids
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [ ]:
# Getting a Glove Model
model_files = ["new.glove.twitter.27B.25d.txt", "new.glove.twitter.27B.50d.txt", 
               "new.glove.twitter.27B.100d.txt", "new.glove.twitter.27B.200d.txt"]
list_of_num_features = [25, 50, 100, 200]

languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age", "gender"]
scoring_function = 'accuracy'
all_results = {}

In [7]:
model_file = "new.glove.twitter.27B.25d.txt"
model = gensim.models.Word2Vec.load_word2vec_format(model_file,binary=False)

# ****** Run k-means on the word vectors and print a few clusters
#

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster

avg_word_per_cluster = 100
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / avg_word_per_cluster

# Initalize a k-means object and use it to extract centroids
print "Running K means"
kmeans_clustering = KMeans( n_clusters = num_clusters, n_jobs=cores )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

In [10]:
elapsed/(60*60*24)

4.959446870775686

In [11]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number
word_centroid_map = dict(zip( model.index2word, idx ))

In [12]:
# Print the first ten clusters
for cluster in xrange(0,10):
#
    # Print the cluster number
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

In [13]:
words

[u'treadmil',
 u'toplantimiz',
 u'selanjutny',
 u'selanjut',
 u'kedpn',
 u'kalix',
 u'brsamaan',
 u'kaliny',
 u'ptama',
 u'brulang',
 u'mjls',
 u'kdatangan',
 u'prtmo',
 u'\u4e88\u7d04\u6570',
 u'prtandingan',
 u'neveruary',
 u'pertame',
 u'mnggal',
 u'semntara',
 u"d'dunia",
 u'yaumil',
 u'brmula',
 u'tdie',
 u'ptma',
 u'mthri',
 u'kdpn',
 u'smpena',
 u'petama',
 u'dhari',
 u'bersm',
 u'slnjtnya',
 u'pertma',
 u'klinya',
 u'keduo',
 u'blnan',
 u'prtemuan',
 u'meninggl',
 u'\u0628\u0627\u0644\u0641\u0644\u0648\u0628\u0627\u0643\u0640',
 u'qiyam',
 u'bkumpul',
 u'prtmuan',
 u'tahn',
 u'daraja',
 u'nh\xfck\xfcmet',
 u'\u307f\u304f\u3067\u3059',
 u'berakhr',
 u'crah',
 u'slanjutnya',
 u'ksekian',
 u'berikutny',
 u'kedpan',
 u'brkhir',
 u'kdpan',
 u'diangka',
 u'diinfo',
 u'brikutnya',
 u'\u062e\u0640\u0627\u0635',
 u'tyng',
 u'brktnya',
 u'n\u5c0f\u8aac',
 u'kdepan',
 u'prtm',
 u'mpkt',
 u'ditgl',
 u'tnyt',
 u'mlwn',
 u'mlahirkan',
 u'ssdh',
 u'playanan',
 u'brikut',
 u'brtahun',
 u"kali'

In [15]:
#saving kmeans model
from sklearn.externals import joblib
joblib.dump(kmeans_clustering, 'kmeans-div-100.pkl')

['kmeans-div-100.pkl',
 'kmeans-div-100.pkl_01.npy',
 'kmeans-div-100.pkl_02.npy']

In [16]:
new_clf = joblib.load('kmeans-div-100.pkl') 